In [1]:
%load_ext autoreload
%autoreload 2

## Part II: Model Building

Here you try your hand at model building to predict appointment no shows.

### Preprocessing

Package 'noshow_lib' now includes code to carry out preprocessing steps from part I. Here's how to use it:

In [2]:
import noshow_lib.util as utils

/usr/local/lib/python2.7/dist-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


First, it includes a dictionary used for configuring path and file names
used through the project

In [3]:
utils.file_config

{'feature_pipeline_file': 'feature_pipeline.pkl',
 'labels_pipeline_file': 'labels_pipeline.pkl',
 'objstore_path': 'objects',
 'processed_data_path': 'processed_data',
 'raw_data_csv': 'KaggleV2-May-2016.csv',
 'raw_data_path': 'data',
 'test_csv': 'test_set.csv',
 'train_csv': 'train_set.csv'}

`feature_pipeline_file`: file storing the preprocessing pipeline used for preparing the feature matrix

`labels_pipeline_file`: file storing the preprocessing pipeline used for
preparing labels

`objstore_path`: directory to store python objects to disk

`processed_data_path`: directory containing processed data

`raw_data_csv`: name of the csv download from Kaggle

`raw_data_path`: directory containing raw data

`test_csv`: name of csv file containing test set

`train_csv`: name of csv file containing train set

You can change these paths and names to suit your project directory structure if you need so. E.g.,

In [4]:
file_config = utils.file_config
#config['raw_data_path'] = "some_other_directory"

First step is to create train test sets. Code is in file `noshow_lib/util.py` function `make_train_test_sets`. You
can edit that function as needed to include your own part I code if you so desire. The result will be to 
create files `train_set.csv` and `test_set.csv` in your `processed_data` directory (unless you change any of the entries in the configuration directory as above).

In [5]:
# ONLY NEED TO RUN THIS STEP ONCE (switch this to True to run it)
RUN_MAKE_TRAIN_TEST_FILES = True
if RUN_MAKE_TRAIN_TEST_FILES:
    utils.make_train_test_sets(config=file_config)

Next step is to fit the preprocessing pipelines. This is done in file `noshow_lib/preprocess.py`. Again you can edit code as needed in that file to incorporate your part I solution as you wish. The result will be to create files `feature_pipeline.pkl` and `labels_pipeline.pkl` containing the fit preprocessing pipelines we can then use to preprocess data.

In [7]:
import noshow_lib.preprocess as preprocess

# ONLY NEED TO RUN THIS STEP ONCE
RUN_FIT_PREPROCESSING = True
if RUN_FIT_PREPROCESSING:
    preprocess.fit_save_pipelines(config=file_config)

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/_encoders.py:326: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)]'.
  warnings.warn(msg, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/_encoders.py:326: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)]'.
  warnings.warn(msg, DeprecationWarning)


Finally, once we do that, we can get a training matrix and labels:

In [8]:
train_X, train_y = preprocess.load_train_data(config=file_config)

In [9]:
print(train_X.shape)
print(train_y.shape)

(90526, 101)
(90526,)


### Model Building

Using `sklearn` fit:
    - DecisionTree classifier
    - RandomForest classifier
    - Linear SVM classifier
    - SVM with Radial Basis Kernel classifier
    
Use default parameters for now.
Using 10-fold cross validation report both accuracy and AUC for each of the above four models.

QUESTION: Should you use accuracy or AUC for this task as a performance metric?

_ANSWER HERE_

All 4 classifiers seem pretty bad. The original dataset had about 79% of patients showing up for appointments. Having a model that just predicts false (no shows) for all patients would do better than these models. Similarly the AUC values are bad, close to 0.5, meaning just making random guesses would work better.  

I would probably choose an AUC here. It would give me the info needed to pick a threshold that would maximize my TPR. In this case it would allow doctors to double book people who they suspect will be no shows. Sure the FPR would be higher, but that just means people will have to wait a bit longer to see the doctor, which already happens these days.

In [19]:
# build your models here
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

# Decision Tree
scores_acc = cross_val_score(DecisionTreeClassifier(), train_X, train_y, cv=10, scoring='accuracy')
scores_auc = cross_val_score(DecisionTreeClassifier(), train_X, train_y, cv=10, scoring='roc_auc')

In [21]:
print scores_acc
print scores_auc

[ 0.74273721  0.74185353  0.74240583  0.74229537  0.73434221  0.73831879
  0.74038886  0.73254529  0.74480778  0.73077773]
[ 0.58589999  0.58666476  0.58376334  0.58661971  0.57711156  0.57962615
  0.58986953  0.57428182  0.58709045  0.5765555 ]


In [22]:
from sklearn.ensemble import RandomForestClassifier

# Random Forrest
scores_acc = cross_val_score(RandomForestClassifier(), train_X, train_y, cv=10, scoring='accuracy')
scores_auc = cross_val_score(RandomForestClassifier(), train_X, train_y, cv=10, scoring='roc_auc')

/usr/local/lib/python2.7/dist-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [23]:
print scores_acc
print scores_auc

[ 0.77808461  0.77698001  0.77786369  0.77631724  0.77554402  0.77289296
  0.77872293  0.77364118  0.77574017  0.77121078]
[ 0.70032626  0.69681271  0.68820308  0.69846838  0.69151386  0.69314235
  0.68807828  0.68562313  0.69791322  0.6874412 ]


In [24]:
from sklearn.svm import LinearSVC

# Linear SVM
scores_acc = cross_val_score(LinearSVC(), train_X, train_y, cv=10, scoring='accuracy')
scores_auc = cross_val_score(LinearSVC(), train_X, train_y, cv=10, scoring='roc_auc')

/usr/local/lib/python2.7/dist-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [25]:
print scores_acc
print scores_auc

[ 0.79608969  0.79509555  0.79675246  0.79597923  0.79586877  0.796642
  0.79595669  0.79750331  0.79639859  0.79617764]
[ 0.66531013  0.66873271  0.67112264  0.66972307  0.6617669   0.67428816
  0.66172633  0.66432048  0.67135942  0.66482576]


In [26]:
from sklearn.svm import SVC

# Linear SVM with Radial-basis function kernel 
scores_acc = cross_val_score(SVC(), train_X, train_y, cv=10, scoring='accuracy')
scores_auc = cross_val_score(SVC(), train_X, train_y, cv=10, scoring='roc_auc')

/usr/local/lib/python2.7/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [27]:
print scores_acc
print scores_auc

[ 0.79807799  0.79807799  0.79807799  0.79807799  0.79807799  0.79807799
  0.79805568  0.79805568  0.79805568  0.79805568]
[ 0.5663538   0.56630337  0.62730793  0.60272141  0.60423637  0.56051229
  0.57808653  0.55330968  0.60036892  0.56649478]


### Model Tuning

Based on the above, choose two methods and fit a tuned model:
    - use 5-fold cross validation for model selection
    - use 10-fold cross validation for model assessment (based on appropriate performance metric)

Report estimated performance for both tuned classifiers

I pick the linear svm and random forrest.  

Not really sure what I'm expected to do here? Just pick two models and run 5,10-fold cross validation?

In [144]:
# tune your models here

# Linear SVM
scores_acc_linsvm = cross_val_score(LinearSVC(tol=1e-1,C=0.01,loss='hinge',verbose=0), train_X, train_y, cv=5, scoring='accuracy')
scores_auc_linsvm = cross_val_score(LinearSVC(tol=1e-1,C=0.01,loss='hinge',verbose=0), train_X, train_y, cv=5, scoring='roc_auc')

In [145]:
print scores_acc_linsvm
print scores_auc_linsvm

[ 0.79807799  0.79806683  0.79806683  0.79806683  0.79806683]
[ 0.47331098  0.4790839   0.47082755  0.55893769  0.54792773]


In [146]:
# random forest
scores_acc_rfc = cross_val_score(RandomForestClassifier(n_estimators=100,verbose=0), train_X, train_y, cv=10, scoring='accuracy')
scores_auc_rfc = cross_val_score(RandomForestClassifier(n_estimators=100,verbose=0), train_X, train_y, cv=10, scoring='roc_auc')

In [147]:
print scores_acc_rfc
print scores_auc_rfc

[ 0.78217166  0.77852646  0.78062521  0.78084613  0.7789683   0.78217166
  0.78071144  0.77982766  0.78038003  0.77375166]
[ 0.7288729   0.7196642   0.71923103  0.72693162  0.71778251  0.72097548
  0.72239428  0.71278933  0.72189748  0.71107458]


### Linear SVM with Gradient Descent

In [336]:
import numpy as np

# Equation looks like = sum (1-y*(w*x+b)) + lambda*w^2
#
# initialize model parameters w and b
# intializing to 0 is not a good idea
# it should be a random vector see np.random.randn
# YOU NEED TO IMPLEMENT THIS
def _initialize_parameters(nfeatures):
    w = np.random.randn((nfeatures))
    b = 0#np.random.rand(1)
    return w, b

# this is a vectorized version of positive_part operation
# we can use this for hinge loss as post_part(1.0 - y*f)
pos_part = np.vectorize(lambda u: u if u > 0. else 0.)

# compute the value of the linear SVM objective function
# given current signed distances, and parameter vector w
def _get_objective(f, y, w, lam):
    loss = np.sum(pos_part(1.0 - y*f))
    penalty = lam * np.dot(w,w)
    return loss + penalty

# compute the signed distances
# based on current model estimates
# w and b
# YOU NEED TO IMPLEMENT THIS
def _get_signed_distances(X, w, b):
    nobs = X.shape[0]
    f = np.full(nobs, 0.0)
    for i,obs in enumerate(X):
        f[i] = b + np.dot(obs,w)
    return f

# compute gradients with respect to w and b
# YOU NEEED TO IMPLEMENT THIS

def _get_gradients(f, X, y, w, b, lam):
    gb = np.sum(-1*y)
    gw = np.zeros(w.shape)
    for y1,obs in zip(y,X):
        gw += -1*obs*y1
    gw += 2*lam*w
    return gw, gb

# fit an SVM using gradient descent
# X: matrix of feature values
# y: labels (-1 or 1)
# lam: penalty parameter lambda
# n_iter: numer of iterations
# eta: learning rate
def fit_svm(X, y, lam, n_iter=100, eta=.4):
    nexamples, nfeatures = X.shape
    
    w, b = _initialize_parameters(nfeatures)
    
    for k in range(n_iter):
        f = _get_signed_distances(X, w, b)
        
        # print information and 
        # update the learning rate
        if k % 10 == 0:
            obj = _get_objective(f, y, w, lam)
            eta = eta / 2.0
            print("it: %d, obj %.2f" % (k, obj))
        
        gw, gb = _get_gradients(f, X, y, w, b, lam)
        w = w - eta * gw
        b = b - eta * gb
    return w, b

In [337]:
w,b = fit_svm(train_X, train_y, 1.0, n_iter=100)

it: 0, obj 176934.46
it: 10, obj 3724833172.83
it: 20, obj 4726749739.43
it: 30, obj 5221209644.65
it: 40, obj 5468094381.32
it: 50, obj 5591493508.10
it: 60, obj 5653185280.22
it: 70, obj 5684029505.77
it: 80, obj 5699451234.87
it: 90, obj 5707162007.18
